In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [34]:
people_df = pd.read_csv("NHANES_data_train.csv")
trainDf, testDf = train_test_split(people_df, test_size = 0.2, random_state = None)

MIcount = (trainDf["MI"] == 1).sum()
noMIcount = (trainDf["MI"] == 2).sum()
testMIcount = (testDf["MI"] == 1).sum()
testnoMIcount = (testDf["MI"] == 2).sum()

dirtyTrainMI_sample = trainDf[trainDf["MI"] == 1]
cleanTrainNoMISample = ((trainDf[trainDf["MI"] == 2]).dropna())
noMIcount = (cleanTrainNoMISample["MI"] == 2).sum()
cleanTrainNoMISample = cleanTrainNoMISample.sample(frac=(MIcount/noMIcount))
    
dirtyTestMI_sample = testDf[testDf["MI"] == 1]
cleanTestNoMISample = ((testDf[testDf["MI"] == 2]).dropna())

imputer = KNNImputer(n_neighbors = 3)
    
crudeCleanTrainMI = imputer.fit_transform(dirtyTrainMI_sample)
cleanTrainMI = pd.DataFrame(crudeCleanTrainMI, columns = dirtyTrainMI_sample.columns)

crudeCleanTestMI = imputer.fit_transform(dirtyTestMI_sample)
cleanTestMI = pd.DataFrame(crudeCleanTestMI, columns = dirtyTestMI_sample.columns)

fTrainSample = (pd.concat([cleanTrainMI, cleanTrainNoMISample])).sample(frac=1)
fTestSample = (pd.concat([cleanTestMI, cleanTestNoMISample])).sample(frac=1)

xTrain = fTrainSample.drop("MI", axis = 1)
yTrain = fTrainSample["MI"]

xTest = fTestSample.drop("MI", axis=1)
yTest = fTestSample["MI"]  # Corrected to use fTestSample


lr = LogisticRegression(random_state= 0)
scaler = StandardScaler()
pipeline = make_pipeline(scaler, lr)
pipeline.fit(xTrain, yTrain)

yPred = pipeline.predict(xTest)

from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(yTest, yPred))
print(classification_report(yTest, yPred))

0.7198581560283688
              precision    recall  f1-score   support

         1.0       0.29      0.81      0.42        36
         2.0       0.96      0.71      0.81       246

    accuracy                           0.72       282
   macro avg       0.62      0.76      0.62       282
weighted avg       0.88      0.72      0.76       282
